In [ ]:
from shutil import copy2
import mikeplus as mp

db_original = "Dyrup_uncalibrated.sqlite"
db_path = "Dyrup_uncalibrated_ex5_copy.sqlite"

copy2(db_original, db_path)

In [ ]:
conc_times_sec = [1200, 1500, 1800, 2100, 2400]
r_factors = [0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
with mp.open(db_path) as db:

    alt_group = db.alternative_groups["Catchments and hydrology data"]

    for tc, r in zip(conc_times_sec, r_factors):

        name = f"Trial (tc={tc:.0f}, r={r:.3f})"
        alternative = alt_group.create(name)
        scenario = db.scenarios.create(name)
        scenario.set_alternative(alternative)
        scenario.activate()

        (
            db.tables.msm_Catchment
                .update({
                    db.tables.msm_Catchment.columns.ModelAConcTime : tc,
                    db.tables.msm_Catchment.columns.ModelARFactor : r
                })
                .all()
                .execute()
        )

        (
            db.tables.msm_Project
                .update({
                    db.tables.msm_Project.columns.ScenarioName : scenario.name
                })
                .by_muid("rainfall")
                .execute()
        )

        result_files = db.run("rainfall")

        print(scenario)
        print(result_files)

